## Assignment:
### Abhishek Sen Roll : 03
#### 1)finding closure
#### 2)finding all the candidate keys
#### 3)finding all the candidate keys
#### 4)finding minimal cover of functional dependancies
#### 5)checking normal forms

here each functional dependancy is considered as a list: <br>
AB -> C  =>    ["AB","C"]<br>
the array FD[][] is a 2dimentional array containing all the Functional Dependancies:<br>
AB->C    <br>
B->D   <br>
equivalent FD array :-<br>
[["AB","C"],["B","D"]]




In [1]:
#closure
def closure(attributes,FD):
    closure_set=""
    l=len(FD)
       
    #the closure will always contain the attribute itself
    closure_set+=attributes
    
    len_bfr,len_after=0,0
    i=0
    
    while(True):
        len_bfr=len(closure_set)
        #going over the Functional Dependancies multiple number of times till a complete execution does not add 
        #anything new to the set
        for i in range(l):

            satisfy=1
            #going through all the LHS components of the the functional dependancy in consideration 
            for j in FD[i][0]:
                #if any LHS attribute of the functional dependancy is absent
                if closure_set.find(j)<0:
                    satisfy=0
            if satisfy==1:
                closure_set+=FD[i][1]

            #removing redundant elements by breaking into a set and rejoining
            closure_set="".join(set(closure_set))

            
            
        len_after=len(closure_set)
        if(len_bfr== len_after): break
        
    return closure_set
        
        
        
    

In [2]:
FD=[["A","BC"],["AB","DE"],["E","FG"],["CG","H"]]
attribute="EC"
setOfAttributes="ABCDEFGH"

print(closure(attribute,FD))

CEHFG


In [3]:
import itertools
def candidate_key(FD,set_of_attributes):
    #finding out the attributes which are not in RHS by initializing and subtracting one FD by one 
    attributes_not_present_in_RHS = set_of_attributes
    
    #set subtracting for removing RHS attributes
    for i in FD:
        attributes_not_present_in_RHS="".join(set(attributes_not_present_in_RHS)-set(i[1]))
    
    candidate_keys = []
    
    
    attributes_in_RHS="".join(set(set_of_attributes)-set(attributes_not_present_in_RHS))
    
    #as the attribute which is not in RHS will always be part of the candidate key
    key=attributes_not_present_in_RHS
    
    #if those attribute which are not in RHS form the candidate key then no other candidate key is possible
    if set(closure(key,FD))==set(set_of_attributes):
        candidate_keys.append(key)
        return candidate_keys #as no other candidate key is present
    
    
    for i in range(len(attributes_in_RHS)):
        
        #finding out combinations of all lengths one by one and checking
        combinations=["".join(i) for i in list(itertools.combinations(attributes_in_RHS,i))]
        
        
        for j in combinations:
            key=attributes_not_present_in_RHS+j
            
            #removing redundant elements
            key="".join(set(key))
            #checking if key is super key
            if set(closure(key,FD))==set(set_of_attributes):
                flag_candidate_key=1
                #checking if the candidate keys already found is a subset of the present key(then the present key is a superkey not candidate key)
                for ky in candidate_keys:
                    if set(ky).issubset(set(key)) :
                        flag_candidate_key=0
                
                if(flag_candidate_key==1):
                    candidate_keys.append(key)
                    
    return candidate_keys

In [4]:
#testing
#FD=[["AB","BC"],["AB","DE"],["E","FG"],["CG","H"]]
#FD=[["AB","C"],["C","A"]]
FD=[["A","BC"],["CD","E"],["B","D"],["E","A"]]
attribute="E"
setOfAttributes="ABCDE"
#setOfAttributes="ABC"
print(candidate_key(FD,setOfAttributes))

['A', 'E', 'BC', 'CD']


In [5]:
def equivalent_functional_dependencies(FD_1,FD_2):
    
    #checking if FD1 subset of FD2
    FD1_C_FD2=1
    for i in range(len(FD_1)):
        #checking closure of each Functional dependancy in FD1 with respect of FD2
        closure_set=closure(FD_1[i][0],FD_2)
        if not set(FD_1[i][1]).issubset(set(closure_set)):
            FD1_C_FD2=0
            break
            
    #checking is FD2 is candidate key is subset of FD1
    FD2_C_FD1=1
    for i in range(len(FD_2)):
        #checking closure of each Functional dependancy in FD2 with respect of FD1
        closure_set=closure(FD_2[i][0],FD_1)
        if not set(FD_2[i][1]).issubset(set(closure_set)):
            FD2_C_FD1=0
            break
            
    if FD2_C_FD1==1 and FD1_C_FD2==1:
        return True
    else :
        return False
    
    

In [6]:
def canonical_cover(FD):
    #step1 producing singleton rhs
    singleton_RHS=[]
    for i in range (len(FD)):
        for j in FD[i][1]:
            singleton_RHS.append([FD[i][0],j])
         
    #step2 removing redundant FDs
    i=0
    while(i<len(singleton_RHS)):
        #poping each functional dependancy
        fd=singleton_RHS.pop(i)
        #checking closure without that functional dependancy
        closure_set=closure(fd[0],singleton_RHS)
        if(not set(fd[1]).issubset(set(closure_set))):
            singleton_RHS.insert(i,fd)
            i+=1
    
    
    return singleton_RHS

In [7]:
#testing
print(canonical_cover([["X","W"],["WZ","XY"],["Y","WXZ"]]))

[['X', 'W'], ['WZ', 'Y'], ['Y', 'X'], ['Y', 'Z']]


In [8]:
#if attribute is a proper subset of any of the candidate keys
def is_prime_attribute(attribute,candidate_keys):
    for ck in candidate_keys:
        if set(attribute).issubset(set(ck)) and set(attribute)!=set(ck):
            return True
    return False


def is_super_key(superkey,candidate_keys):
    for ck in candidate_keys:
        if set(ck).issubset(set(superkey)):
            return True
    return False

In [9]:
#testing
FD=[["AB","BC"],["AB","DE"],["E","FG"],["CG","H"]]
#FD=[["AB","C"],["C","A"]]
attribute="B"
set_of_attributes="ABCDEFGH"
#setOfAttributes="ABC"
cks=candidate_key(FD,set_of_attributes)
print(is_prime_attribute(attribute,cks))

True


In [10]:
#checking if FDs are in 2NF
def two_nf(FD,set_of_attributes):
    #finding set of candidate keys
    candidate_keys=candidate_key(FD,set_of_attributes)
    for fd in FD:
        #checking if partial dependancy is present (LHS is prime)(RHS is not prime) 
        if is_prime_attribute(fd[0],candidate_keys) and not is_prime_attribute(fd[1],candidate_keys):
            return False
        
    print("two_nf")
    return True

In [11]:
#checking if FDs are in 3NF
def three_nf(FD,set_of_attributes):
    #FDs has to be in second normal form
    if not two_nf(FD,set_of_attributes):
        return False
    #finding set of candidate keys
    candidate_keys=candidate_key(FD,set_of_attributes)
    for fd in FD:
        #LHS has to be super key or RHS has to be prime attribute
        if not(is_super_key(fd[0],candidate_keys)  or is_prime_attribute(fd[1],candidate_keys) ):
            return False
    print("three_nf")
    return True

In [12]:
#checking if FDs are in BCNF
def bcnf(FD,set_of_attributes):
    #FDs need to be in 3NF
    if not three_nf(FD,set_of_attributes):
        return False
    candidate_keys=candidate_key(FD,set_of_attributes)
    for fd in FD:
        #LHS fas to be super key 
        if not(is_super_key(fd[0],candidate_keys)):
            return False
    print("bcnf")
    return True

In [14]:
#RUN code from here(run all cells above before executing this)
set_of_attributes=input("enter the set of attributes without space: ")
n=int(input("enter the number of functional dependancies:  "))
print("Enter the FDs in the form - LHS->RHS  (no spaces)")
FD=[]
for i in range(n):
    s=input()
    fd=[s[0:s.find('-')],s[s.find('>')+1:]]
    FD.append(fd)
    
attribute=input("enter attribute whose closure has to be found: ")
print("closure",closure(attribute,FD))
print("highest normal form(all the normal forms will be printed as they will get checked)",bcnf(FD,set_of_attributes))
print("canonical cover",canonical_cover(FD))

n1=int(input("enter the number of functional dependancies(for equivalence checking):  "))
print("Enter the FDs in the form - LHS->RHS  (no spaces): ")



FD1=[]
for i in range(n1):
    s=input()
    fd=[s[0:s.find('-')],s[s.find('>')+1:]]
    FD1.append(fd)
    
if(equivalent_functional_dependencies(FD,FD1)):
    print("the FDs are equivalent")
else:
    print("the FDs are not equivalent")


enter the set of attributes without space: ACDEH
enter the number of functional dependancies:  4
Enter the FDs in the form - LHS->RHS  (no spaces)
A->C
AC->D
E->AD
E->H
enter attribute whose closure has to be found: A
closure ACD
two_nf
highest normal form(all the normal forms will be printed as they will get checked) False
canonical cover [['A', 'C'], ['AC', 'D'], ['E', 'A'], ['E', 'H']]
enter the number of functional dependancies(for equivalence checking):  2
Enter the FDs in the form - LHS->RHS  (no spaces): 
A->CD
E->AH
the FDs are equivalent


In [249]:
FD=[["A","BC"],["CD","E"],["AB","D"],["E","A"]]
superkey="CDB"
set_of_attributes="ABCDE"
cks=candidate_key(FD,set_of_attributes)
print(cks)

print(bcnf(FD,set_of_attributes))

['A', 'E', 'CD']
['A', 'E', 'CD']
two_nf
three_nf
bcnf
True
